AI-Powered Financial Analysis: Multi-Agent LLM Systems Transform Data into Insights

 
Batuhan Sener
12 min read
Jul 30, 2024
 
Financial Agents with GPT-4o-mini and LLaMA 3.1 and more … Click here to explore …
This article shares the experiences of developing a multi-agent LLM system that gathers relevant information from the internet, performs sentiment analysis on comments from Reddit, conducts fundamental and technical analysis, and summarizes the information about stock, using a language model. The system is made accessible through Streamlit.
Overview of Essential Technologies and Frameworks
Before starting the project, I would like to talk about the technologies and frameworks I used in this project:
•	1-) CrewAI : Multi-agent system.
•	2-) Langchain : Tools & model usage.
•	3-) Reddit(praw) & Yahoo Finance(yfinance): Data collecting.
•	4-) Hugging Face : Finetuned sentiment model for finance.
•	5-) Groq Cloud and OpenAI : Accessing LLama and GPT models.
•	6-) Streamlit : UI & Deployment.
Explaining Crew AI and How I Used It
Crew AI is a collaborative working system designed to enable various artificial intelligence agents to work together as a team, efficiently accomplishing complex tasks.
Includes crew ai agent tasks and tools.
Agents :Each agent is an AI unit that takes on a specific role and works according to that role. Every agent is designed to achieve a specific goal and can perform certain tasks.
Tasks: Tasks are specific assignments that agents need to complete, along with instructions on how to complete them. Tasks define the expected outcomes and the tools that will be used.
Tools: Tools are additional resources and integrations that agents can use while performing their tasks. These tools provide functions like data collection, analysis, or access to specific APIs.
In this project, I integrated Meta and OpenAI’s language models. I created custom tools and developed agents such as Researcher, Technical and Fundamental Analyst, and Reporter to gather stock information. I will share my experiences with this crew system.
Explaining Langchain and How I Used It
LangChain is an open-source framework for building applications based on large language models (LLMs). It provides tools and abstractions to customize, improve the accuracy, and relevance of the information generated by the models. Developers can use LangChain components to create new prompt chains or customize existing templates. LangChain also allows LLMs to access new data sets without retraining.
In this project, I used the YahooFinanceNewsTool for stock information and accessed Groq and OpenAI models.
Data Collecting from Reddit and Yahoo Finance
PRAW (Python Reddit API Wrapper) is a library used for easy access to Reddit’s API. PRAW provides a user-friendly interface for fetching Reddit data, browsing subreddits, creating posts and comments, and interacting with Reddit.
YFINANCE is a Python library used to fetch financial data from Yahoo Finance. It allows you to easily obtain various financial data such as stock prices, historical data, financial reports, and other financial information.
In this project, I used yfinance to collect data for fundamental and technical analysis and for visualization. I used praw to fetch data from Reddit to analyze users’ opinions about the stock from the ‘wallstreetbets’, ‘stocks’, and ‘investing’ subreddits.
Explaining Hugging Face and How I Used It
Hugging Face is a company and an open-source platform that provides tools and resources for building, training, and deploying machine learning models. The Hugging Face Hub hosts many models for a variety of machine learning tasks.
In this project, I used the Hugging Face model “distilroberta-finetuned-financial-news-sentiment-analysis” to analyze the sentiment of data from Reddit.
Groq and OpenAI
Groq is a technology company that develops innovative hardware and software solutions for machine learning and artificial intelligence. Their products are designed to provide high-performance, low-latency computing for complex AI workloads.
In this project, I integrated GPT-4o, GPT-4o-mini, LLaMA 3 8B, LLaMA 3.1 8B, and LLaMA 3.1 70B language models from Groq Cloud and OpenAI Platform.
Streamlit UI and Cloud
Streamlit is an open-source framework used for creating interactive web applications specifically for machine learning, data science and LLM projects. Streamlit Cloud is a platform for deploying, managing, and sharing applications directly from GitHub repository.
In my project, I used Streamlit to create an interactive interface and deployed it on Streamlit Cloud.
Steps Covered in this Tutorial
The steps to be examined for the multi-agent system:
•	Custom Tools for Financial Analysis
•	Defining the Crew
•	Developing the Streamlit Application & Deployment the App
•	Project Usage and Output
To keep the article concise, I will not delve into the detailed explanation of all the code. For more information, feel free to contact me. My contact details will be shared at the end of the page.
1- Custom Tools for Financial Analysis
Tools are specialized components that agents can use to perform specific tasks more effectively. These tools can provide various functions such as data collection, analysis, interaction with APIs, or accessing specific resources. You can even integrate it with Langchain tools.
In this article, I will explain the Sentiment tool, but I will also include descriptions of the other tools I used.
•	Browser Tool: Implementing web browsing functionality.
•	Search Tools: Developing specific search operations with SERPER.
•	Sentiment Analysis Tool: Building a tool to analyze sentiment in financial data.
•	Fundamental Analysis Tool: Creating a tool for fundamental stock analysis using Yahoo Finance.
•	YahooFinanceNewsTool: Gathering and analyzing stock-related news from Langchain’s tool.
•	Tech Analysis Tool: Implementing technical stock analysis using Yahoo Finance.

Sentiment Analysis Tool :
First I loads the model and tokenizer from the transformers library for sentiment analysis. The analyze_sentiment function then performs sentiment analysis on given text and returns the result as "negative," "neutral," or "positive." The get_reddit_posts function retrieves posts containing a specific stock symbol from a particular subreddit. Using the praw library, it connects to the Reddit API and fetches the posts. These posts are filtered to include only those from the last 30 days. The ‘ @ tool’ decorator allows the function to be used as a CrewAI tool. The reddit_sentiment_analysis function performs sentiment analysis on posts from specified subreddits about a stock symbol and returns the count of each sentiment label.


In [ ]:
# sentiment_analysis_tool.py
import os
import praw
import torch
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from crewai_tools import tool

# Download hf model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

def analyze_sentiment(text):
    """
    Analyze the sentiment of a given text.
    """
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    scores = outputs.logits.softmax(dim=1).detach().numpy()[0]
    labels = ["negative", "neutral", "positive"]
    label = labels[scores.argmax()]
    return label

def get_reddit_posts(subreddit_name, stock_symbol, limit=100, days=30):
    """
    Get posts from a specific subreddit containing the stock symbol within the last specified days.
    """
    reddit = praw.Reddit(
        client_id=os.getenv("REDDIT_CLIENT_ID"),
        client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
        user_agent=os.getenv("REDDIT_USER_AGENT")
    )
    subreddit = reddit.subreddit(subreddit_name)
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=30)
    
    posts = []
    for post in subreddit.search(stock_symbol, sort='new', time_filter='month', limit=limit):
        post_date = datetime.utcfromtimestamp(post.created_utc)
        if start_date <= post_date <= end_date:
            posts.append(post.title)
    return posts

@tool
def reddit_sentiment_analysis(stock_symbol: str, subreddits: list = ['wallstreetbets', 'stocks', 'investing'], limit: int = 100):
    """
    Perform sentiment analysis on posts from specified subreddits about a stock symbol.
    
    Args:
        stock_symbol (str): The stock symbol to search for.
        subreddits (list): List of subreddits to search in.
        limit (int): Number of posts to fetch from each subreddit.
    
    Returns:
        list: List of sentiment labels for each post.
    """
    all_sentiments = []
    sentiments_counts={'neutral': 0, 'negative': 0, 'positive': 0}
    
    for subreddit in subreddits:
        posts = get_reddit_posts(subreddit, stock_symbol, limit)
        for post in posts:
            sentiment = analyze_sentiment(post)
            all_sentiments.append((sentiment))
            sentiments_counts[sentiment]+=1

    return sentiments_counts


Defining the Crew
For model selection, I created a function called initialize_llm using LLaMA or OpenAI, and wrote the create_crew function to define the tools to be used. I defined Agents and Tasks. When creating agents, I ensured clear goals and detailed background information to make them more capable and specific. For tasks, I provided clear instructions and expected outputs to make them more understandable and achievable. Using the Crew class, I integrated all agents and tasks to form and start the team. Finally, I saved the results to a file and returned them to the user.


In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from tools.sentiment_analysis_tool import reddit_sentiment_analysis
from tools.yf_tech_analysis_tool import yf_tech_analysis
from tools.yf_fundamental_analysis_tool import yf_fundamental_analysis
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Environment Variables
load_dotenv()
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["REDDIT_CLIENT_ID"] = os.getenv("REDDIT_CLIENT_ID")
os.environ["REDDIT_CLIENT_SECRET"] = os.getenv("REDDIT_CLIENT_SECRET")
os.environ["REDDIT_USER_AGENT"] = os.getenv("REDDIT_USER_AGENT")

# Model Selection
def initialize_llm(model_option, openai_api_key, groq_api_key):
    if model_option == 'OpenAI GPT-4o':
        return ChatOpenAI(openai_api_key=openai_api_key, model='gpt-4o', temperature=0.1)
    elif model_option == 'OpenAI GPT-4o Mini':
        return ChatOpenAI(openai_api_key=openai_api_key, model='gpt-4o-mini', temperature=0.1)
    elif model_option == 'Llama 3 8B':
        return ChatGroq(groq_api_key=groq_api_key, model='llama3-8b-8192', temperature=0.1)
    elif model_option == 'Llama 3.1 70B':
        return ChatGroq(groq_api_key=groq_api_key, model='llama-3.1-70b-versatile', temperature=0.1)
    elif model_option == 'Llama 3.1 8B':
        return ChatGroq(groq_api_key=groq_api_key, model='llama-3.1-8b-instant', temperature=0.1)
    else:
        raise ValueError("Invalid model option selected")


def create_crew(stock_symbol, model_option, openai_api_key, groq_api_key):
    llm = initialize_llm(model_option, openai_api_key, groq_api_key)
    # Tools Initialization
    reddit_tool = reddit_sentiment_analysis
    serper_tool = SerperDevTool()
    yf_tech_tool = yf_tech_analysis
    yf_fundamental_tool = yf_fundamental_analysis

    # Agents Definitions
    researcher = Agent(
        role='Senior Stock Market Researcher',
        goal='Gather and analyze comprehensive data about {stock_symbol}',
        verbose=True,
        memory=True,
        backstory="With a Ph.D. in Financial Economics and 15 years of experience in equity research, you're known for your meticulous data collection and insightful analysis.",
        tools=[reddit_tool, serper_tool, YahooFinanceNewsTool()],
        llm=llm
    )

    technical_analyst = Agent(
        role='Expert Technical Analyst',
        goal='Perform an in-depth technical analysis on {stock_symbol}',
        verbose=True,
        memory=True,
        backstory="As a Chartered Market Technician (CMT) with 15 years of experience, you have a keen eye for chart patterns and market trends.",
        tools=[yf_tech_tool],
        llm=llm
    )

    fundamental_analyst = Agent(
        role='Senior Fundamental Analyst',
        goal='Conduct a comprehensive fundamental analysis of {stock_symbol}',
        verbose=True,
        memory=True,
        backstory="With a CFA charter and 15 years of experience in value investing, you dissect financial statements and identify key value drivers.",
        tools=[yf_fundamental_tool],
        llm=llm
    )

    reporter = Agent(
        role='Chief Investment Strategist',
        goal='Synthesize all analyses to create a definitive investment report on {stock_symbol}',
        verbose=True,
        memory=True,
        backstory="As a seasoned investment strategist with 20 years of experience, you weave complex financial data into compelling investment narratives.",
        tools=[reddit_tool, serper_tool, yf_fundamental_tool, yf_tech_tool, YahooFinanceNewsTool()],
        llm=llm
    )

    # Task Definitions
    research_task = Task(
        description=(
            "Conduct research on {stock_symbol}. Your analysis should include:\n"
            "1. Current stock price and historical performance (5 years).\n"
            "2. Key financial metrics (P/E, EPS growth, revenue growth, margins).\n"
            "3. Recent news and press releases (1 month).\n"
            "4. Analyst ratings and price targets (min 3 analysts).\n"
            "5. Reddit sentiment analysis (100 posts).\n"
            "6. Major institutional holders and recent changes.\n"
            "7. Competitive landscape and {stock_symbol}'s market share.\n"
            "Use reputable financial websites for data."
        ),
        expected_output='A detailed 150-word research report with data sources and brief analysis.',
        agent=researcher
    )

    technical_analysis_task = Task(
        description=(
            "Perform technical analysis on {stock_symbol}. Include:\n"
            "1. 50-day and 200-day moving averages (1 year).\n"
            "2. Key support and resistance levels (3 each).\n"
            "3. RSI and MACD indicators.\n"
            "4. Volume analysis (3 months).\n"
            "5. Significant chart patterns (6 months).\n"
            "6. Fibonacci retracement levels.\n"
            "7. Comparison with sector's average.\n"
            "Use the yf_tech_analysis tool for data."
        ),
        expected_output='A 100-word technical analysis report with buy/sell/hold signals and annotated charts.',
        agent=technical_analyst
    )

    fundamental_analysis_task = Task(
        description=(
            "Conduct fundamental analysis of {stock_symbol}. Include:\n"
            "1. Review last 3 years of financial statements.\n"
            "2. Key ratios (P/E, P/B, P/S, PEG, Debt-to-Equity, etc.).\n"
            "3. Comparison with main competitors and industry averages.\n"
            "4. Revenue and earnings growth trends.\n"
            "5. Management effectiveness (ROE, capital allocation).\n"
            "6. Competitive advantages and market position.\n"
            "7. Growth catalysts and risks (2-3 years).\n"
            "8. DCF valuation model with assumptions.\n"
            "Use yf_fundamental_analysis tool for data."
        ),
        expected_output='A 100-word fundamental analysis report with buy/hold/sell recommendation and key metrics summary.',
        agent=fundamental_analyst
    )

    report_task = Task(
        description=(
            "Create an investment report on {stock_symbol}. Include:\n"
            "1. Executive Summary: Investment recommendation.\n"
            "2. Company Snapshot: Key facts.\n"
            "3. Financial Highlights: Top metrics and peer comparison.\n"
            "4. Technical Analysis: Key findings.\n"
            "5. Fundamental Analysis: Top strengths and concerns.\n"
            "6. Risk and Opportunity: Major risk and growth catalyst.\n"
            "7. Reddit Sentiment: Key takeaway from sentiment analysis, including the number of positive, negative and neutral comments and total comments.\n"
            "8. Investment Thesis: Bull and bear cases.\n"
            "9. Price Target: 12-month forecast.\n"
        ),
        expected_output='A 600-word investment report with clear sections, key insights.',
        agent=reporter
    )

    # Crew Definition and Kickoff for Result
    crew = Crew(
        agents=[researcher, technical_analyst, fundamental_analyst, reporter],
        tasks=[research_task, technical_analysis_task, fundamental_analysis_task, report_task],
        process=Process.sequential,
        cache=True
    )

    result = crew.kickoff(inputs={'stock_symbol': stock_symbol})

    os.makedirs('./crew_results', exist_ok=True)
    file_path = f"./crew_results/crew_result_{stock_symbol}.md"
    result_str = str(result)
    with open(file_path, 'w') as file:
        file.write(result_str)
    
    return file_path


Developing the Streamlit Application & Deployment the App
I created an interactive area that allows the user to select a model, integrates API keys for model usage, and generates charts using Yahoo Finance data. The multi-agent system performs all the steps and displays the outputs on the screen. Afterwards, I uploaded the code to GitHub and integrated it with Streamlit Cloud. To keep this article concise, I am concluding the explanation here. For a more detailed code review, please contact me privately.

Project Usage and Output
•	First, select the model you want to use.
•	To run the project, you need to obtain an API key based on the model you select. I have shared a link on how you can access Groq and OpenAI APIs.
•	Enter the stock symbol you are interested in for a detailed analysis like (NVDA, AAPL, ACN)
•	For interactive charts, select the time period and indicators, then click the “Analyze Stock” button.
The model will then gather data about the selected stock from the internet and provide you with an analysis based on various parameters, including fundamental and technical analysis outputs and sentiment results, according to the template of the chosen model.
Overview :
 
 
Output :
Stock Information
Company Name: NVIDIA Corporation
Sector: Technology
Industry: Semiconductors
Country: United States
Current Price: $103.73
Market Cap: $2551581769728
Analysis Result

NVIDIA Corporation (NVDA) Investment Report

1. Executive Summary: Investment Recommendation NVIDIA Corporation (NVDA) is a leading player in the semiconductor industry, particularly known for its graphics processing units (GPUs) that power gaming, AI, and data center applications. Given its robust financial health, strong growth metrics, and competitive positioning, the recommendation is to Hold. While the stock appears oversold, it shows potential for recovery amidst current bearish trends.

2. Company Snapshot: Key Facts

Company Name: NVIDIA Corporation
Sector: Technology
Industry: Semiconductors
Market Cap: $2.55 trillion
P/E Ratio: 60.66
PEG Ratio: 0.89
Debt to Equity Ratio: 22.87
3. Financial Highlights: Top Metrics and Peer Comparison NVIDIA's financial performance is characterized by:

Revenue Growth (YoY): 25.85%
Net Income Growth (YoY): 58.13%
Gross Margin: 75.29%
Operating Margin: 64.93%
Net Profit Margin: 53.40% Compared to peers, NVDA's high P/E ratio indicates strong growth expectations, while its low PEG ratio suggests it may be undervalued relative to its growth potential.
4. Technical Analysis: Key Findings

Current Price: $103.73
20-Day Moving Average: $122.04
50-Day Moving Average: $119.38
RSI: 21.04 (indicating oversold conditions)
MACD: Bearish
Support Level: $102.54
Resistance Level: $136.15 The technical indicators suggest a bearish breakdown, but the stock is currently oversold, indicating a potential for a price rebound.
5. Fundamental Analysis: Top Strengths and Concerns Strengths:

Strong revenue and net income growth rates.
Low debt-to-equity ratio, indicating financial stability.
High gross and operating margins, reflecting efficient operations.
Concerns:

High P/E ratio may indicate overvaluation.
Current bearish trends in the market could impact short-term performance.
6. Risk and Opportunity: Major Risk and Growth Catalyst Major Risk: The semiconductor industry is highly cyclical and sensitive to economic downturns, which could adversely affect NVDA's sales and profitability.

Growth Catalyst: The increasing demand for AI and machine learning applications presents a significant growth opportunity for NVIDIA, as its GPUs are essential for these technologies.

7. Reddit Sentiment: Key Takeaway from Sentiment Analysis From the sentiment analysis conducted on Reddit:

Total Comments Analyzed: 100
Positive Comments: 3
Negative Comments: 6
Neutral Comments: 24 The sentiment is predominantly neutral, with a slight inclination towards negativity, reflecting cautious investor sentiment amidst current market conditions.
8. Investment Thesis: Bull and Bear Cases Bull Case: If NVIDIA continues to capitalize on the growing AI market and maintains its competitive edge, it could see substantial revenue growth, leading to a higher stock price.

Bear Case: Economic headwinds and increased competition in the semiconductor space could lead to declining sales and profitability, negatively impacting the stock price.

9. Price Target: 12-Month Forecast Considering the current market conditions and NVDA's growth potential, a conservative 12-month price target is set at $130, reflecting a recovery from current oversold levels and aligning with historical performance trends.

In conclusion, while NVIDIA Corporation presents a compelling investment opportunity due to its strong fundamentals and growth potential, investors should remain cautious given the current market sentiment and technical indicators. A hold position is recommended until clearer bullish signals emerge.

Conclusion and Recommendations
•	It is not investment advice. Just personal hobby :)
•	It’s important to keep the descriptions for Agents and Tasks short and concise. Long and detailed descriptions can exceed the Tokens-per-Minute (TPM) limit and be very costly.
•	Before creating the fundamental and technical analysis tools, I was completing tasks by fetching all the data from the internet. This resulted in a high processing load, so I created custom tools to meet this need. This approach is more cost-effective. It’s important to assess needs properly.
•	GPT models have shown better inferences compared to LLaMA models. GPT-4o-mini is very functional. It has a higher TPM limit compared to GPT-4, and the results are good. It’s a more cost-effective model.
References
Reddit (wallstreetbets, stocks, investing)
Yahoo Finance
CrewAI Docs
Streamlit Docs
Langchain Tools
You can use the link to try the project
All codes are my Github
If you would like to consult or get in touch, you can reach me via LinkedIn or email.
If you enjoy my content, please consider showing your support by clapping for my articles or buying me a coffee. Your support is greatly appreciated.
